Of Interest:

Attempting to train the model using both the available GPUs

In [15]:
import numpy as np
import torch 
import torchvision
import torch.nn as nn
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.utils.data import *
from torch.optim import Adam
from torch.optim import lr_scheduler
from __future__ import print_function, division
import time
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import torch.nn.parallel as parallel

In [2]:
x_data = np.load('x_data.npy')
y_data = np.load('y_data.npy')

In [16]:
# Download and load pretrained resnet.
vgg16 = torchvision.models.vgg16(pretrained=True)

In [4]:
vgg16.classifier[6].out_features = 15

In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('cuda available')
    vgg16 = parallel.DataParallel(vgg16).cuda()

cuda available


In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
# Observe that only fc parameters are being optimized
optimizer = Adam(vgg16.parameters(), lr=0.001)

In [8]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
class CustomDataSet(Dataset):
    """Dataset wrapping data and target tensors.
    
    Each sample will be retrieved by indexing both tensors along the first
    dimension.

    Arguments:
        data_tensor (Tensor): contains sample data.
        target_tensor (Tensor): contains sample targets (labels).
    """

    def getCategoryLabel(self, y_vec):
        tot = y_vec.shape[0]
        vec = y_vec.shape[1]

        y_cat = np.zeros(tot)
        for i in range(tot):
            for j in range(vec):
                if (y_vec[i,j] == 1.0):
                    y_cat[i]=j
                    break                       
        return y_cat    
    
    def __init__(self, data_arr, target_arr):
        assert data_arr.shape[0] == target_arr.shape[0]
        self.data_arr = data_arr
        self.target_arr = torch.from_numpy(self.getCategoryLabel(target_arr).astype('uint8'))

        self.transform = transforms.Compose([
            transforms.Scale((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])            
        ])
        
    def __getitem__(self, index):
        
        # this hack needed because transform will not act on images directly!
        # Have to first convert to img using Image.fromarray(..)
        # The use the transforms from above
        temp_img = Image.fromarray(self.data_arr[index].astype('uint8'))
        mod_img = self.transform(temp_img)

        label = self.target_arr[index]        
        return mod_img, label

    def __len__(self):
        return self.data_arr.shape[0]


In [10]:
# I need to be sure to randomize x_data and y_data, because the below only takes the last 3000 snapshots
# of the track, which isn't ideal way to validate...
train_data = CustomDataSet(x_data[:-3000], y_data[:-3000])
valid_data = CustomDataSet(x_data[-3000:], y_data[-3000:])

In [11]:
dataset_sizes={}
dataset_sizes['train'] = len(train_data)
dataset_sizes['valid'] = len(valid_data)

In [12]:
dataLoaders = {}
dataLoaders['train'] = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=8)
dataLoaders['valid'] = DataLoader(valid_data, batch_size=32, shuffle=True, num_workers=8)

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in tqdm(dataLoaders[phase]):
                # get the inputs
                inputs, labels = data
                
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [14]:
train_model(vgg16, criterion, optimizer, exp_lr_scheduler)

  0%|          | 0/117 [00:00<?, ?it/s]

Epoch 0/24
----------


  0%|          | 0/94 [00:00<?, ?it/s]

train Loss: 0.0223 Acc: 0.3027


  0%|          | 0/117 [00:00<?, ?it/s]

...

train Loss: 0.0103 Acc: 0.4908


  0%|          | 0/117 [00:00<?, ?it/s]

valid Loss: 0.0392 Acc: 0.5173

Epoch 5/24
----------


 96%|█████████▌| 112/117 [01:34<00:04,  1.18it/s]Process Process-84:
KeyboardInterrupt
Process Process-85:
Process Process-83:
Process Process-86:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-82:
Traceback (most recent call last):
Process Process-81:
Process Process-88:
Traceback (most recent call last):
  File "/home/apil/anaconda3/envs/pytorch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/apil/anaconda3/envs/pytorch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/apil/anaconda3/envs/pytorch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/apil/anaconda3/envs/pytorch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/apil/anaconda3/envs/pytorch/lib/python

KeyboardInterrupt: 